In [1]:
import import_ipynb
from phase1_edge_handling import edge_builder_agent

d:\agentagent\AgentAgent\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
input_dict = {
    "conditional_rand" : """
Write a conditional edge from node: src to one of the nodes ["node1", "node2", "node3"]
The condition is use a random number to select any of the node""",
"deterministic_parallel": """Add edge from node1 to node 2 and node 3""",
"deterministic_1": "Add edge from node 1 to node2",
"conditional_state_tool_call": "Write a conditional_edge, if the the last message in messages field had a tool_call, then route to tools node, else END",
"code_and_edge_non_conditional":
    """
<GraphNodeImplementation>
def plan_step(state: OverallState) -> OverallState:
    \"\"\"Plan step generates a plan based on the input using llm structured output functionality, stores it to the plan field\"\"\"
    
    # Extract the input from the state
    user_input = state['input']
    
    # Initialize the structured LLM with the Plan schema
    structured_llm = llm.with_structured_output(Plan)
    
    # Generate the structured plan using the input
    plan: Plan = structured_llm.invoke(
        f"Please create a structured plan based on the following input: {{user_input}}. "
        "The plan should clearly outline the steps to take in order to achieve the requested outcome, "
        "and provide detailed information for each step in a format that includes titles and descriptions."
    )
    
    # Update the state with the generated plan
    state['plan'] = [{'title': title, 'description': description} for title, description in zip(plan.titles, plan.descriptions)]
    
    return state
</GraphNodeImplementation>

<EdgeInformation>
"source": "planner",
      "target": "agent",
      "routing_conditions": "After planning, execute the first step.",
      "conditional": false
</EdgeInformation>
    """,
    "supervisor": """
<GraphNodeImplementation>
class Router(TypedDict):
    \"\"\"Worker to route to next. If no workers needed, route to FINISH.\"\"\"
    next: Literal[*options]
    action: str = Field("the action to be taken by the next worker")

class State(MessagesState):
    next: str
    
members = ["food_logger", "workout_tracker"]
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = members + ["FINISH"]


def supervisor(state: State):
    system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    f" following workers: {{members}}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
    )

    llm = ChatAnthropic(model="claude-3-5-sonnet-latest")
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    return {"next": goto, "action": response["action"]}
    </GraphNodeImplementation>
    <EdgeInformation>
    { 
      "source": "supervisor", 
      "target": "food_logger", 
      "target_info": "uses state action in its implementation",
      "routing_conditions": "If the supervisor decides to route to food_logger based on user input.", 
      "conditional": true 
    },
    { 
      "source": "supervisor", 
      "target": "workout_tracker",
      "target_info": "uses state action in its implementation",
      "routing_conditions": "If the supervisor decides to route to workout_tracker based on user input.", 
      "conditional": true 
    },
    { 
      "source": "supervisor", 
      "target": "__END__", 
      "routing_conditions": "If the supervisor decides to finish the process.", 
      "conditional": true
    }
    </Edgeinformation>
    """
}


In [5]:
import uuid
uuid = uuid.uuid4()
config = {"configurable": {"thread_id": str(uuid)}}

result = edge_builder_agent.invoke({"messages": [HumanMessage(content = input_dict["supervisor"])]},config)

In [ ]:
from pprint import pprint
pprint(result["messages"][-1].content)